In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data2/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data2/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['0','1282','label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 

dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
df= dtlatih('latih_parted') #df.X,df.Y,df.dt

#RF

In [3]:
def rf(n,m,df):
    #random_grid = {'n_estimators': [n],
   #                #'max_depth': max_depth,
    #               'bootstrap': [True],
    #               'max_samples':[m],}
    rf = BaggingClassifier(n_estimators=n,bootstrap=True,max_samples=m,base_estimator=DecisionTreeClassifier(class_weight="balanced"))
    #rf = GridSearchCV(rf, random_grid, n_jobs=-1, cv=2)
    rf.fit(df.X, df.Y)
    filename = '../data2/model/rf_{0}n_{1}m_balanced'.format(n,m)
    pickle.dump(rf, open(filename, 'wb'))
    return rf

#MLP

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

Using TensorFlow backend.


In [147]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='normal', activation='relu'))
    model.add(Dense(320, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [155]:
def mlp(n,m,df):
    mlp = MLPClassifier(max_iter=1000, batch_size=128,hidden_layer_sizes=(640,320,),activation= 'relu',
                        solver= 'adam', alpha= 0.5,learning_rate= 'constant')
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='uniform', activation='relu'))
    model.add(Dense(320, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=128, verbose=0,class_weight="balanced")))
    pipeline = Pipeline(estimators)
#    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#    results = cross_val_score(pipeline, df.X, df.Y, cv=kfold)
#    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
#    parameter_space = {
#        'hidden_layer_sizes': [(640,320,)],
#        'activation': ['relu'],
#        'solver': ['adam'],
#        'alpha': [0.5],
#        'learning_rate': ['constant'],
#    }
    mlp.fit(df.X, df.Y,class_weight="balanced")
    filename = '../data2/model/mlp_{0}n_{1}m_balanced'.format(n,m)
    pickle.dump(mlp, open(filename, 'wb'))
    return mlp

In [156]:
fitn2pred(mlp,10,100,0.5,df,dtf,dtl)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


n = 10
m = 100
model file : mlp_10n_100m_balanced
Parameters used: {'activation': 'relu', 'alpha': 0.5, 'batch_size': 128, 'beta_1': 0.9, 'beta_2': 0.999, 'early_stopping': False, 'epsilon': 1e-08, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'learning_rate_init': 0.001, 'max_iter': 1000, 'momentum': 0.9, 'n_iter_no_change': 10, 'nesterovs_momentum': True, 'power_t': 0.5, 'random_state': None, 'shuffle': True, 'solver': 'adam', 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': False, 'warm_start': False}
Accuracy: 0.9918806232170287
f1: 0.9727740986019131
precision: 0.9821693907875185
recall: 0.9635568513119533
roc_auc: 0.9802284385725358
Confusion matrix:
[[3859   12]
 [  25  661]]
Results on the train set:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3871
           1       0.98      0.96      0.97       686

    accuracy                           0.99      4557
   macro avg       0.99      0.98      0.9

In [133]:
#SVM

In [5]:
def svm1(n,m,df):
    classifier0 = SVC(C=8.0, gamma=0.0078125,class_weight="balanced",probability=True)
    classifier0.fit(df.X, df.Y)
    filename = '../data2/model/svm1_{0}n_{1}m_balanced'.format(n,m)
    pickle.dump(classifier0, open(filename, 'wb'))
    return classifier0

In [6]:
def svm2(n,m,df):
    classifier0 = SVC(C=0.125,kernel='poly' ,gamma=0.0078125,class_weight="balanced",probability=True)
    classifier0.fit(df.X, df.Y)
    filename = '../data2/model/svm2_{0}n_{1}m_balanced'.format(n,m)
    pickle.dump(classifier0, open(filename, 'wb'))
    return classifier0

In [7]:
def svm3(n,m,df):
    class_weight={ 0:0.59, 1:3.33 }
    classifier0 = SVC(C=8.0,kernel='rbf' ,gamma=0.0078125,class_weight=class_weight,probability=True)
    classifier0.fit(df.X, df.Y)
    filename = '../data2/model/svm3_{0}n_{1}m_balanced'.format(n,m)
    pickle.dump(classifier0, open(filename, 'wb'))
    return classifier0

In [8]:
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2):
    md=svm_model(model,n,m,datalatih)
    prediksi(md,model,n,m,p,datauji1)
    prediksi(md,model,n,m,p,datauji2)

In [14]:
fitn2pred(svm1,10,100,0.5,df,dtf,dtl)

n = 10
m = 100
model file : svm1_10n_100m_balanced
Parameters used: {'C': 8.0, 'break_ties': False, 'cache_size': 200, 'class_weight': 'balanced', 'coef0': 0.0, 'decision_function_shape': 'ovr', 'degree': 3, 'gamma': 0.0078125, 'kernel': 'rbf', 'max_iter': -1, 'probability': True, 'random_state': None, 'shrinking': True, 'tol': 0.001, 'verbose': False}
Accuracy: 0.9659863945578231
f1: 0.8982271831910702
precision: 0.8162291169451074
recall: 0.9985401459854014
roc_auc: 0.979383709356337
Confusion matrix:
[[3718  154]
 [   1  684]]
Results on the train set:
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      3872
           1       0.82      1.00      0.90       685

    accuracy                           0.97      4557
   macro avg       0.91      0.98      0.94      4557
weighted avg       0.97      0.97      0.97      4557

threshold %f 
hasil prediksi  mbafika  : (0,1)
[[199  41]
 [  0   0]]
   0  1               2                 3

In [ ]:
fitn2pred(svm2,0,0,0.5,df,dtf,dtl)

In [ ]:
fitn2pred(svm3,0,0,0.5,df,dtf,dtl)

In [14]:
def mlp_model(model,n,m,data):
    md = mlp(n,m,data)
    report(model,n,m,data)
    return md

In [15]:
def define_model(mod,n,m,data):
    md = (sys.modules[__name__], "%s" % mod.__name__)(n,m,data)
    report(mod,n,m,data)
    return md

In [16]:
def rf_model(model,n,m,data):
    md = rf(n,m,data)
    report(model,n,m,data)
    return md

In [10]:
def svm_model(model,n,m,data):
    md = svm1(n,m,data)
    report(model,n,m,data)
    return md

In [135]:
#SCORE

In [12]:
def report(model,n,m,df):
    rf = pickle.load(open('../data2/model/{0}_{1}n_{2}m_balanced'.format(model.__name__,n,m), 'rb'))
    pred_x = rf.predict(df.X)
    print('n = {0}'.format(n))
    print('m = {0}'.format(m))
    print('model file : {0}_{1}n_{2}m_balanced'.format(model.__name__,n,m))
    print('Parameters used:', rf.get_params() )
    #print('Best parameters found:', rf.best_params_)
    print('Accuracy:',accuracy_score(df.Y, pred_x))
    print('f1:',f1_score(df.Y, pred_x))
    print('precision:',precision_score(df.Y, pred_x))
    print('recall:',recall_score(df.Y, pred_x))
    print('roc_auc:',roc_auc_score(df.Y, pred_x))
    print('Confusion matrix:')
    print(confusion_matrix(df.Y, pred_x))
    print('Results on the train set:')
    print(classification_report(df.Y, pred_x))
    #pred_n = rf.predict(cX)
    #print('Results on the all neg data:')
    #print(classification_report(cY, pred_n))
    

In [13]:
def prediksi(md,model,n,m,p,data):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold %f ".format(p))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    print(pred.head())
    print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data2/hasil_pred/{0}_{1}n_{2}m_{3}p_{4}_balanced.xlsx'.format(model.__name__,n,m,p,data.dt))

In [11]:
import sys



In [12]:
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2):
    md=svm1_model(model,n,m,datalatih)
    prediksi(md,model,n,m,p,datauji1)
    prediksi(md,model,n,m,p,datauji2)

In [173]:
fitn2pred(svm,10,100,0.5,df,dtf,dtl)

AttributeError: module '__main__' has no attribute 'svm_model'

In [45]:
#fitn2pred(rf,100,1000,df,dtf,dtl)

In [46]:
#fitn2pred(rf,300,1000,df,dtf,dtl)

In [47]:
#fitn2pred(rf,500,1000,df,dtf,dtl)

In [48]:
#fitn2pred(rf,1000,1000,df,dtf,dtl)

In [49]:
#fitn2pred(rf,100,2500,df,dtf,dtl)

In [113]:
fitn2pred(rf,300,2500,0.5,df,dtf,dtl)

n = 300
m = 2500
model file : rf_300n_2500m_balanced
Parameters used: {'base_estimator__class_weight': 'balanced', 'base_estimator__criterion': 'gini', 'base_estimator__max_depth': None, 'base_estimator__max_features': None, 'base_estimator__max_leaf_nodes': None, 'base_estimator__min_impurity_decrease': 0.0, 'base_estimator__min_impurity_split': None, 'base_estimator__min_samples_leaf': 1, 'base_estimator__min_samples_split': 2, 'base_estimator__min_weight_fraction_leaf': 0.0, 'base_estimator__presort': False, 'base_estimator__random_state': None, 'base_estimator__splitter': 'best', 'base_estimator': DecisionTreeClassifier(class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=Non

In [114]:
#fitn2pred(rf,500,2500,df,dtf,dtl)

In [115]:
#fitn2pred(rf,1000,2500,df,dtf,dtl)

In [116]:
#fitn2pred(rf,100,3500,df,dtf,dtl)

In [117]:
#fitn2pred(rf,300,3500,df,dtf,dtl)

In [118]:
#fitn2pred(rf,500,3500,df,dtf,dtl)

In [180]:
fitn2pred(rf,1000,3500,0.5,df,dtf,dtl)

n = 1000
m = 3500
model file : rf_1000n_3500m_balanced
Parameters used: {'base_estimator__class_weight': 'balanced', 'base_estimator__criterion': 'gini', 'base_estimator__max_depth': None, 'base_estimator__max_features': None, 'base_estimator__max_leaf_nodes': None, 'base_estimator__min_impurity_decrease': 0.0, 'base_estimator__min_impurity_split': None, 'base_estimator__min_samples_leaf': 1, 'base_estimator__min_samples_split': 2, 'base_estimator__min_weight_fraction_leaf': 0.0, 'base_estimator__presort': False, 'base_estimator__random_state': None, 'base_estimator__splitter': 'best', 'base_estimator': DecisionTreeClassifier(class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=N